In [6]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)
 
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from src.features import build_features

sns.set_theme(palette='pastel')
plt.rcParams['figure.figsize'] = (12, 6)
pd.set_option('display.max_columns', None)

# Complicaciones Quirúrgicas

En este cuadernillo se quieren identificar y cuantificar las complicaciones quirúrgicas en el INT.

In [8]:
RUTA_PABELLON = "../data/external/datos_limpios_pabellon.csv"

# Leer y formatear datos de operaciones quirúrgicas
df_pabellon = pd.read_csv(RUTA_PABELLON, encoding="latin-1", sep=";")
df_pabellon["fecha"] = pd.to_datetime(df_pabellon["fecha"])
df_pabellon["duracion"] = pd.to_timedelta(df_pabellon["duracion"])

# Limpiar datos de operaciones quirúrgicas
df_pabellon_limpio = build_features.limpiar_datos_operaciones_quirurgicas(df_pabellon)
df_pabellon_limpio = df_pabellon_limpio.reset_index(names="id_int_q")

In [9]:
# Define las complicaciones a buscar
a_buscar_operaciones = {
    "ecmo": r"ECMO",
    "trasplantes": r"TRASPLANTE|TRANSPLANTE",
    "aseos": r"ASEO",
    "drenajes": r"DRENAJE",
    "traqueostomias": r"TRAQUEOSTOMIA|TRAQUEOSTOMÍA",
    "reintervenciones_reoperaciones": r"REINTERVENCION|REOPERACION|REINTERVENCIÓN|REOPERACIÓN",
}

# Define los diagnosticos que son complicaciones
a_buscar_diagnosticos = {
    "empiemas": r"EMPIEMA"
}

In [29]:
# Busca los nombres de las operaciones en la base de pabellon
resultados_operaciones = build_features.iterar_en_complicaciones_a_buscar(
    df_pabellon_limpio, a_buscar_operaciones, "intervencion_quirurgica"
)

# Busca los diagnosticos en el primer diagnostico y segundo
resultados_diagnosticos = build_features.iterar_en_complicaciones_a_buscar(
    df_pabellon_limpio, a_buscar_diagnosticos, "diagnostico"
)

# Obtiene resumen de complicaciones
resumen_complicaciones = pd.concat([resultados_operaciones[1], resultados_diagnosticos[1]])

In [30]:
resumen_complicaciones

,especialidad,totales,ocurrencia_filtrado,fraccion,complicacion
0,CT,6491,2.0,0.000308,ecmo
1,CV,6372,242.0,0.037979,ecmo
2,MCP,1433,0.0,0.000000,ecmo
3,TRASPLANTE,210,27.0,0.128571,ecmo
0,CT,6491,0.0,0.000000,trasplantes
1,CV,6372,3.0,0.000471,trasplantes
2,MCP,1433,0.0,0.000000,trasplantes
3,TRASPLANTE,210,100.0,0.476190,trasplantes
0,CT,6491,764.0,0.117701,aseos
1,CV,6372,317.0,0.049749,aseos
